# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
# model_name_or_path = 'THUDM/chatglm3-6b'
# 调试时发现奇怪问题，虽然已经设置huggingface镜像，并且在其他notebook中可以正常运行并下载模型，但是使用model name加载模型时出现huggingface连接超时
# 所以暂时使用两个变量，保证正常运行
model_name = 'THUDM/chatglm3-6b'
model_path = '/root/autodl-tmp/hf/hub/models--THUDM--chatglm3-6b/snapshots/f30825950ce00cb0577bf6a15e0d95de58e328dc'

In [2]:
import os

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HF_HOME'] = '/root/autodl-tmp/hf'
os.environ['HF_HUB_CACHE'] = '/root/autodl-tmp/hf/hub'

In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

2024-02-17 17:37:31.664660: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-17 17:37:31.664718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-17 17:37:31.666491: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-17 17:37:31.677926: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 17:37:32.821923: W tensorflow/comp

In [8]:
print(response)

乾卦是《易经》中的一个卦象，它是由两个阴爻夹一个阳爻构成，象征着天、云、雷等自然现象。乾卦的含义非常丰富，它既象征着阳刚之气、创造力和行动力，也象征着阴柔之美、适应能力和包容力。在《易经》中，乾卦是八卦之一，它与坤卦（地）共同构成了天地之象，象征着天地之间相互依存、相互影响的关系。

乾卦的卦辞描述了它的特性：“元、亨、利、贞。”这四个字意味着：元，表示万物本源，是宇宙创造力的基础；亨，表示事物发展顺利，如道路畅通；利，表示和谐、美好，如收获的果实；贞，表示正道，如事物的规律和秩序。乾卦的意义在于揭示事物发展规律和正道，指导人们做出正确的决策和行为。

乾卦的六爻则代表了不同的时间段和不同的态度。从下往上，每一爻的阳爻代表一个阶段，从初爻到九爻，阳爻的数量逐渐增加，象征着事物的发展和壮大。每一爻的阴爻则代表了不同的态度和行为，从初爻到九爻，阴爻的数量逐渐减少，象征着事物的逐渐完善和提升。通过研究乾卦的六爻，我们可以了解事物的发展规律，学会在不同阶段采取不同的策略，达到最佳的结果。


In [9]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是《易经》中的一个卦象，它是八卦之一。这个卦象由两个阴爻夹一个阳爻构成，象征着地水相交的自然现象。地水师卦的含义非常丰富，它既象征着阴柔之美、适应能力和包容力，也象征着阳刚之气、创造力和行动力。在《易经》中，地水师卦是八卦之一，它与乾卦（天）和坤卦（地）共同构成了天地之象，象征着天地之间相互依存、相互影响的关系。

地水师卦的卦辞描述了它的特性：“元、亨、利、贞。”这四个字意味着：元，表示万物本源，是宇宙创造力的基础；亨，表示事物发展顺利，如道路畅通；利，表示和谐、美好，如收获的果实；贞，表示正道，如事物的规律和秩序。地水师卦的意义在于揭示事物发展规律和正道，指导人们做出正确的决策和行为。

地水师卦的六爻则代表了不同的时间段和不同的态度。从下往上，每一爻的阳爻代表一个阶段，从初爻到九爻，阳爻的数量逐渐增加，象征着事物的发展和壮大。每一爻的阴爻则代表了不同的态度和行为，从初爻到九爻，阴爻的数量逐渐减少，象征着事物的逐渐完善和提升。通过研究地水师卦的六爻，我们可以了解事物的发展规律，学会在不同阶段采取不同的策略，达到最佳的结果。


## 微调前后效果对比

In [10]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name}-epoch{epochs}"

In [11]:
print(peft_model_path)

models/THUDM/chatglm3-6b-epoch50


In [12]:
config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

ValueError: Can't find 'adapter_config.json' at 'models/THUDM/chatglm3-6b-epoch50'

### 使用微调后的 ChatGLM3-6B

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [15]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [16]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B (Epoch=3, automade-dataset) 微调后：\n{ft_response}")
    return base_response, ft_response

In [17]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。卦辞为“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

在哲学和卜筮方面，乾卦代表着刚健、积极、勇敢的特性，同时提示着要遵守天道的德行。通过观察乾卦的卦象，人们可以学会自强不息，永不放弃，坚持自己的信念和目标。同时，乾卦也提醒人们，在追求成功的过程中，要勇敢地面对困难和挑战，并坚持到底。

在风水和卜卦方面，人们认为乾卦能够带来吉祥如意，驱散邪气，带来好运。因此，在生活中遇到困难时，可以通过风水调整和卜卦来寻求解决办法。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。卦辞云：“元、亨、利、贞”，预示着吉祥如意，同时也教导人们遵守天道的德行。乾卦所蕴含的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

在哲学和卜卦方面，乾卦代表着刚健、强劲、运行不止的特性，同时也预示着忧患和困难。通过观察乾卦的卦象，可以得出天道的运行规律，从而预测未来的发展趋势。同时，乾卦也强调自强不息，运行不已，因此，在解卦时，要注重观象取义，以天为法，吉祥如意。


In [18]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是一个由三个卦组成的卦象，分别是地卦、水卦和师卦。在周易中，卦象是用来预测事物走向的一种方法，而卦辞则包含了卦象的详细解释。地卦为坤卦，代表地，象征大地万物，具有承载万物的特性。水卦为坎卦，代表水，象征流动性，具有流动性的一面。师卦为坤卦，代表众，象征众人心意。

地水师卦的卦象为：上地，下坎，中师。这种卦象预示着稳定和承载能力，具有温和、宁静的特性。在预测中，地水师卦象征着事业顺利，生意兴隆，人际关系和善，provided they are led by wisdom and compassion.

地水师卦的核心哲学是：柔胜刚，故为贵。这个哲学意味着，在处理问题和决策时，应该注重柔和、温和的态度，以及适应能力和象征性的思考。

地水师卦的预测范围很广，包括事业、经商、求名、婚恋等方面。根据不同的卦象组合，地水师卦可以分为复、周、远等不同含义，预示着不同的事情。

地水师卦是一个富有哲学意义的卦象，需要结合具体的问题和情况来解释和应用。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由三个卦象组成的卦卦，分别是地卦、水卦和风卦。在周易中，卦卦是预测卦象的符号，而卦象则是由卦卦组成的。地卦为坤卦，水卦为坎卦，风卦为乾卦。在周易中，坤卦为地卦，坎卦为水卦，乾卦为风卦。

在卜筮时，地卦表示顺顺利利，表示地卦预示着顺利的预兆。而水卦表示渡河浮沉，表示水卦预示着困难重重的预兆。风卦表示强劲有力，表示风卦预示着 strong 力量。

地水师卦的卦象为：下卦为坤卦（地卦），上卦为坎卦（水卦）。在卜筮时，地卦表示遇事顺利，而水卦表示困难重重。

地水师卦的核心哲学是：事物之间存在相互影响和相互制约的关系。在事业、经商、求名、婚恋等方面都有具体的应用，需要根据实际情况灵活运用。

地水师卦的预测结果为：若占问国，国全运顺。若占问财，初期艰难，但会得到财富。若占问身，身强体健。若占问婚，婚姻顺利。

地水师卦的建议为：地卦表示遇事顺利，要求遇事 calm、稳重、宽容大度，欢迎他人。而水卦表示困难重重，要求避免冒险，遇到问题要冷静思考，不可冒险。

地水师卦的解释为：地卦代表地，象征事物的发展需要稳定的基础，需要遵循规律和常理。而水卦

In [19]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
{'name': '讼卦"上卦为乾(天),下卦为坎(水)",二者相背而运,天与水违行的状况相同。讼卦是一个阳卦和阴卦相互争斗的卦象,具有很强的变性和动变特性。在卜卦时,讼卦表示出初爻为阴, rest 为阳, rest 刚强指数较大,需要退让,求得解决之道。讼卦的意义为,当阳刚之强强欲争,刚强不柔弱,则守正道,必欲退让,乃可望生。', 'content': '\n讼卦的卦象表明,当双方都欲争取胜利,相互争斗时,很难达成和平的结果。在决策时,应该根据实际情况,权衡利弊,退让或进攻。对于卦象,需要根据上下卦的组合,以及卦象所代表的含义,进行综合分析,以得出正确的判断。\n\n讼卦的特性是难以达成和平的结果,需要退让或进攻。在决策时,需要根据实际情况,权衡利弊,退让或进攻。对于卦象,需要根据上下卦的组合,以及卦象所代表的含义,进行综合分析,以得出正确的判断。'}


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 周易中的讼卦是什么周易中的讼卦是指卦象为上卦坎（水）下卦坤（地），两者相交形成地水相克的现象。在卜卦时，讼卦表示双方势力相当，难以相容，争端有可能发生。对于卜者来说，应该谨慎处理，以避免灾祸。讼卦的卦辞为：“天行健，君子以自强不息。”

讼卦的哲学意义是：势力相当的两方，相互交错，相互压抑，双方难以相容，必然产生争端。在争端中，双方必须保持冷静，以理性和智慧解决问题，不可盲目行事。

讼卦的时运是：利于求名，利于涉水，利于涉地，不利于居丧

讼卦的卦象为：上卦坎（水），下卦坤（地），地水相克的现象。

讼卦的象义是：双方势力相当，相互交错，相互压抑，双方难以相容，必然产生争端。在争端中，双方必须保持冷静，以理性和智慧解决问题，不可盲目行事。

讼卦的解卦是：复卦，卦象为上卦坎（水）下卦坤（地），解卦意味着双方在争端中达成妥协，恢复原状。解卦预示着，在遇到困难时，以冷静、理智的态度寻求解决问题的关键，最终实现问题解决。

讼卦的旅卦是：风卦，卦象为上卦坤（地），下卦风卦（木），旅卦意味着旅行时遇到困难和挫折，彼此相互支持，共同解决问题。旅卦预示着，在生活和工作中遇到困难和挫折时，需要互相帮助，共同解决问题。

讼卦的传统解卦是：复卦。复卦是讼卦的解卦，意味着双方在

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [20]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240217_152507"
peft_model_path = f"models/{model_name}-epoch{epochs}-{timestamp}"

In [21]:
config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [22]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [23]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦，是周易中的一卦，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。', 'content': '\n乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。\n\n在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [24]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
{'name': '卜卦问事，古代以占卜未来为主，现代则以预测现在和未来为主，可参考下述方法：', 'content': '\n1. 象数卦：以卦象来预测，由卜卦者自行其卦象，如乾为天，坤为地，震为雷，巽为风，坎为水，离为火，艮为山，兑为泽。每卦都有其卦象和卦辞，可以参考。\n2. 卜卦问事：卜卦者向宇宙寻求答案，回答其关心的问题。卜卦者可以选择一个或多个卦象，来预测未来。卦象可由卜卦者自行其卦象，也可由预测者代为选择。\n3. 占卜未来：占卜者关心的问题未来的发展如何，可以通过卜卦来预测。卜卦者可以选择一个或多个卦象，来预测未来。卜卦者需将卦象和卦辞告诉回答者，回答者根据卦象和卦辞分析卦象对问事的影响和吉凶。\n4. 象数卦：象数卦是一种通过卦象来预测 method，由卜卦者自行其卦象，如乾为天，坤为地，震为雷，巽为风，坎为水，离为火，艮为山，兑为泽。每卦都有其卦象和卦辞，可以参考。\n\n无论哪种方法，都需要对卦象和卦辞有深入的了解和领悟，才能准确地分析卦象对问事的影响和吉凶。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 是一条线段，从下到上连结而成。它的卦象是由两个卦相叠而成，上面的卦是乾卦，下面的卦是坤卦。乾卦代表天，坤卦代表地，因此地水师卦象征着天地的相融，象征着吉祥。

在这个卦象中，君子观此卦象，应当以天地的合一为原则来处理事情，以实现美好的结果。由于地水师卦是一卦之上的卦，因此，它预示着吉祥如意的结果。同时，也暗示着顺利地完成任务，顺利解决困难。

地水师卦的卦象解读，预示着吉祥如意的结果，由于天地的合一，因此得以实现美好的结果。在事业、经商、婚姻等方面，都有良好的运势。在遇到困难时，也有天助之利，可望解决。地水师卦的核心哲学是：得配天时，得水助，顺地势，利见。


In [25]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
干的卦象为天,上的卦象为地,下的是水。它是纯阳纯阴的卦象,纯阳表示阳盛,纯阴表示阴盛。在这个卦象中,地下的一卦为乾,地上的一卦为坤。乾代表天,坤代表地。天象卦象像天而在地之上,地象卦象像地而在天之下。 both 讼卦和天卦、坤卦和地卦相互关联,形成一个完整的故事。它们揭示了君子观卦象以明辨是非,判断善恶,取舍得宜,得到人心。 讼卦揭示了君子观卦象,以明辨是非,判断善恶,取舍得宜,得到人心。它告诉君子在上位时要宽容谦恭,对待在下位时要公正无私,才能获得吉运。同时,也提醒君子做事要公正、公平,才能得到佳运。 讼卦的卦象特点如下:上卦为天,卦象如天而在地之上;下卦为地,卦象如地而在天之下。卦象呈现卦象相互交织的形态,犹如诉讼诉讼之象,因此得名。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么概念 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。
